In [1]:
import bs4
import random
from lxml import etree

import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from time import sleep

from webdriver_manager.chrome import ChromeDriverManager

In [14]:
def review_details(dom,main_dict,current_page):
    country_list=[i for i in dom.xpath("//div[@class='c-review-block']//div[1]//div[@class='c-review-block__row c-review-block__guest']//div[@class='bui-avatar-block__text']//span[@class='bui-avatar-block__subtitle']//text()") if i!='\n']
    for count,i in enumerate(dom.xpath("//div[@class='c-review-block']")):
        
        try:
            country=country_list[count]
        except:
            country=""
        inner_dict={}
        #Left Block
        
        try:
            name=i.findall("div")[1].findall("div//span[@class='bui-avatar-block__title']")[0].text
        except:
            name=""
        
        try:
            room_type=i.findall("div")[1].findall("div")[0].findall("div[@class='c-review-block__row c-review-block__room-info-row']//ul/li//a//div")[0].text
        except:
            room_type=""
        
        try:
            nights_stayed=i.findall("div")[1].findall("div")[0].findall("ul[@class='bui-list bui-list--text bui-list--icon bui_font_caption c-review-block__row c-review-block__stay-date']//li//div")[0].text
        except:
            nights_stayed=""
            
        try:
            month_stayed=i.findall("div")[1].findall("div")[0].findall("ul[@class='bui-list bui-list--text bui-list--icon bui_font_caption c-review-block__row c-review-block__stay-date']//li//div//span")[0].text
        except:
            month_stayed=""
            
            
        try:
            occupant_type=i.findall("div")[1].findall("div")[0].findall("ul[@class='bui-list bui-list--text bui-list--icon bui_font_caption review-panel-wide__traveller_type c-review-block__row']//li//div")[0].text
        except:
            occupant_type=""
        
        try:
            date_reviewed=i.findall("div")[1].findall("div//span[@class='c-review-block__date']")[0].text
        except:
            date_reviewed=""
        
        try:
            review_score=i.findall("div")[1].findall("div")[1].findall("div")[0].findall("div[@class='bui-grid']")[0] \
              .findall("div[@class='bui-grid__column-1 bui-u-text-right']//div//div")[0].text
        except:
            review_score=""
        
        #Right Block
        
        try:
            short_review=i.findall("div")[1].findall("div//h3")[0].text
        except:
            short_review=""
            
        try:
            positive_review=i.findall("div")[1].findall("div")[1].findall("div")[1].findall("div//div[@class='c-review__row']" +
                            "//p//span[@class='c-review__body']")[0].text
        except:
            positive_review=""

        try:
            negative_review=i.findall("div")[1].findall("div")[1].findall("div")[1].findall("div//div[@class='c-review__row lalala']" +
                         "//p//span[@class='c-review__body']")[0].text
        except:
            negative_review=""

        inner_dict={"name":name, "room_type":room_type, "nights_stayed":nights_stayed, "month_stayed":month_stayed,
                    "occupant_type":occupant_type, "from_country":country,"review_score":review_score,
                   "date_reviewed":date_reviewed,"short_review":short_review,"positive_review":positive_review,
                    "negative_review":negative_review
                   }

        main_dict["From Page "+str(current_page)+", Person- "+ str(count+1)]=inner_dict




def get_review_body(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    driver.maximize_window()


    time.sleep(3)
    driver.execute_script('window.scrollTo(0, 100)')
        
    #click Read all reviews button
    

    
    
    driver.find_element(by="xpath", value="//a[@data-target= 'hp-reviews-sliding']").click()
    #Switch to review window
    driver.switch_to.window(driver.window_handles[0])

    soup = BeautifulSoup(driver.page_source,"html.parser")

    #get min and max pages
    current_page = 1
    time.sleep(3)
    
    all_pageList = driver.find_elements(by="xpath",value="//a[contains(@class,'bui-pagination__link')]")

    try:
        last = driver.find_element(by="xpath", value="//*[@id='review_list_page_container']/div[4]/div/div[1]/div/div[2]/div/div[7]/a/span[2]").text
        last2 = last.split(" ")
        last_page = int(last2[1])
    except Exception as e: 
        if len(all_pageList) == 2:
            last = driver.find_element(by="xpath", value="//*[@id='review_list_page_container']/div[4]/div/div[1]/div/div[2]/div/div[2]/a/span[2]").text
            last2 = last.split(" ")
            last_page = int(last2[1]) + 1
        elif len(all_pageList) == 3:
            last = driver.find_element(by="xpath", value="//*[@id='review_list_page_container']/div[4]/div/div[1]/div/div[2]/div/div[3]/a/span[2]").text
            last2 = last.split(" ")
            last_page = int(last2[1]) + 1
        elif len(all_pageList) == 4:
            last = driver.find_element(by="xpath", value="//*[@id='review_list_page_container']/div[4]/div/div[1]/div/div[2]/div/div[4]/a/span[2]").text
            last2 = last.split(" ")
            last_page = int(last2[1]) + 1
        elif len(all_pageList) == 5:
            last = driver.find_element(by="xpath", value="//*[@id='review_list_page_container']/div[4]/div/div[1]/div/div[2]/div/div[5]/a/span[2]").text
            last2 = last.split(" ")
            last_page = int(last2[1]) + 1
        elif len(all_pageList) == 6:
            last = driver.find_element(by="xpath", value="//*[@id='review_list_page_container']/div[4]/div/div[1]/div/div[2]/div/div[6]/a/span[2]").text
            last2 = last.split(" ")
            last_page = int(last2[1]) + 1
        elif len(all_pageList) == 1:
            last_page = current_page
    
    try:
        print("Total Pages- " + str(last_page))
    except Exception as e: 
        last_page=1
        print("Total Pages- " + str(last_page))
    review_body=[]
    main_dict={}
    while current_page <= last_page:
        print("Current Page- " +str(current_page))
        time.sleep(3)
        
        page_source=driver.page_source
        soup=bs4.BeautifulSoup(page_source,'lxml')
        dom = etree.HTML(str(soup))
        
        
        review_details(dom,main_dict,current_page)
        
        time.sleep(3)
        if current_page<last_page:
            while True:
                try:
                    driver.find_element(by="xpath", value="//a[contains(@class, 'pagenext')]").click()
                    break
                except:    
                    print("Click Error-Trying Again")
                    time.sleep(5)
                    pass
                  
        current_page +=1
    
    driver.quit()
    dict_to_df={"url":url,"All_reviews":main_dict}
    df=pd.DataFrame({k: pd.Series([v]) for k,v in dict_to_df.items()})
    return df


# Run on multiple URL's

In [ ]:
from tqdm import tqdm
#df=pd.read_csv("data/all_manila_hotels.csv")
#df=pd.read_csv("data/all_manila_hotels_v2.csv")
df=pd.read_csv("data/consolidated_urls.csv")
links=df.link

df_hotels_consolidated=pd.DataFrame()

start_index=122
end_index=125

for count,i in tqdm(enumerate(links[start_index:end_index])):
    print("Opening link- "+ str(count) + " " + str(i))
    #print(df.hotel_name[count+start_index])
    #try:
    df_to_append=get_review_body(i)
    df_to_append["Hotel_num"]=start_index+count
    df_hotels_consolidated=pd.concat([df_hotels_consolidated,df_to_append])
        
    # except Exception as e: 
    #     print("")
    #     print("Error:")
    #     print(str(e))
    #     print("Skipping due to error")
    #     continue
        
    print(" ")

0it [00:00, ?it/s]

Opening link- 0 https://www.booking.com/hotel/ph/red-planet-aurora-blvd-quezon-city.html?label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-01&checkout=2022-12-02&dest_id=-2437894&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=10&hapos=35&sr_order=popularity&nflt=ht_id%3D204&srpvid=7ca3572c9035002e&srepoch=1669119846&all_sr_blocks=262187102_105120793_0_0_0&highlighted_blocks=262187102_105120793_0_0_0&matching_block_id=262187102_105120793_0_0_0&sr_pri_blocks=262187102_105120793_0_0_0__121520&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
Total Pages- 79
Current Page- 1
Current Page- 2
Current Page- 3
Current Page- 4
Current Page- 5
Current Page- 6
Current Page- 7
Current Page- 8
Current Page- 9
Current Page- 10
Current Page- 11
Current Page- 12
Current Page

1it [09:16, 556.42s/it]

 
Opening link- 1 https://www.booking.com/hotel/ph/bayview-park-manila.html?label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-01&checkout=2022-12-02&dest_id=-2437894&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=12&hapos=37&sr_order=popularity&nflt=ht_id%3D204&srpvid=7ca3572c9035002e&srepoch=1669119846&all_sr_blocks=40251201_260167281_0_1_0&highlighted_blocks=40251201_260167281_0_1_0&matching_block_id=40251201_260167281_0_1_0&sr_pri_blocks=40251201_260167281_0_1_0__251328&from=searchresults#hotelTmpl
Total Pages- 76
Current Page- 1
Current Page- 2
Current Page- 3
Current Page- 4
Current Page- 5
Current Page- 6
Current Page- 7
Current Page- 8
Current Page- 9
Current Page- 10
Current Page- 11
Current Page- 12
Current Page- 13
Current Page- 14
Current Page- 15
Current P

2it [18:17, 547.29s/it]

 
Opening link- 2 https://www.booking.com/hotel/ph/city-garden-grand-makati.html?label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-01&checkout=2022-12-02&dest_id=-2437894&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=13&hapos=38&sr_order=popularity&nflt=ht_id%3D204&srpvid=7ca3572c9035002e&srepoch=1669119846&all_sr_blocks=111863504_352016593_2_2_0&highlighted_blocks=111863504_352016593_2_2_0&matching_block_id=111863504_352016593_2_2_0&sr_pri_blocks=111863504_352016593_2_2_0__358974&from=searchresults#hotelTmpl
Total Pages- 161
Current Page- 1
Current Page- 2
Current Page- 3
Current Page- 4
Current Page- 5
Current Page- 6
Current Page- 7
Current Page- 8
Current Page- 9
Current Page- 10
Current Page- 11
Current Page- 12
Current Page- 13
Current Page- 14
Current Page- 15

In [10]:
df_hotels_consolidated

,url,All_reviews
0,https://www.booking.com/hotel/ph/artemis-place...,"{'From Page 1, Person- 1': {'name': 'Philip', ..."


In [11]:
import pickle


output = open('data/hotel_reviews_121_121.pkl', 'wb')
pickle.dump(df_hotels_consolidated, output)
output.close()

#df.to_csv("data/sample_output_reviews.csv")